<a href="https://colab.research.google.com/github/ImranNust/DeepLearningWithPyTorch/blob/main/Chapter4/Module4_WorkingWithTimeSeries.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1> <center> <b> <u> Introduction </u> </b> </center> </h1>

- In [Module3](https://github.com/ImranNust/DeepLearningWithPyTorch/blob/main/Chapter4/Module3_RepresentingTabularData.ipynb), we learnt about tabular dataset. Every row in the table was independent from the others; their order did not matter. Or, equivalently, there was no column that encoded information about what rows came earlier and what came later.

- In this module, we will talk about another dataset (time series dataset): [a data from a Washington](https://archive.ics.uci.edu/ml/datasets/bike+sharing+dataset), D.C., bike-sharing system reporting the
hourly count of rental bikes in 2011–2012 in the Capital Bikeshare system, along with weather and seasonal information.

- Our goal will be to take a flat, 2D dataset and transform it into a 3D one, as shown below.

![](https://raw.githubusercontent.com/ImranNust/DeepLearningWithPyTorch/main/Images/module44.png)

---

<h2> <center> <b> <u> Adding a Time Dimension </u> </b> </center> </h2>

In the source data, each row is a separate hour of data. We want to change the row-per-hour organization so that we have one axis that increases at a rate of one day per index increment, and another axis that represents the hour of the day (independent of the date). The third axis will be our different columns of data (weather, temperature, and so on).


Let’s load the data (code/p1ch4/4_time_series_bikes.ipynb).

In [1]:
import numpy as np
import torch
try:
  print("[INFO] Required Folder/Files Already Exists...")
  dir_path = "/content/hour-fixed.csv"
  bikes_numpy = np.loadtxt(dir_path,
                         dtype = np.float32,
                         delimiter = ",",
                         skiprows = 1,
                         converters = {1: lambda x: float(x[8:10])})
except:
  print("[INFO] Required Folder/Files Do Not Exist...; therefore, dowonloading it.. cheers...")
  !git clone https://github.com/ImranNust/DeepLearningWithPyTorch
  !mv DeepLearningWithPyTorch/Images/bike-sharing-dataset/hour-fixed.csv /content/
  !rm -rf DeepLearningWithPyTorch
  dir_path = "/content/hour-fixed.csv"
  bikes_numpy = np.loadtxt(dir_path,
                         dtype = np.float32,
                         delimiter = ",",
                         skiprows = 1,
                         converters = {1: lambda x: float(x[8:10])})

 

bikes = torch.from_numpy(bikes_numpy)
print('------------------------------------------------------------------')
print('The downloaded dataset is \n {}'.format(bikes))

[INFO] Required Folder/Files Already Exists...
[INFO] Required Folder/Files Do Not Exist...; therefore, dowonloading it.. cheers...
Cloning into 'DeepLearningWithPyTorch'...
remote: Enumerating objects: 224, done.
remote: Counting objects: 100% (95/95), done.
remote: Compressing objects: 100% (88/88), done.
remote: Total 224 (delta 11), reused 58 (delta 3), pack-reused 129
Receiving objects: 100% (224/224), 38.71 MiB | 14.97 MiB/s, done.
Resolving deltas: 100% (21/21), done.
Checking out files: 100% (121/121), done.
------------------------------------------------------------------
The downloaded dataset is 
 tensor([[1.0000e+00, 1.0000e+00, 1.0000e+00,  ..., 3.0000e+00, 1.3000e+01,
         1.6000e+01],
        [2.0000e+00, 1.0000e+00, 1.0000e+00,  ..., 8.0000e+00, 3.2000e+01,
         4.0000e+01],
        [3.0000e+00, 1.0000e+00, 1.0000e+00,  ..., 5.0000e+00, 2.7000e+01,
         3.2000e+01],
        ...,
        [1.7377e+04, 3.1000e+01, 1.0000e+00,  ..., 7.0000e+00, 8.3000e+01,
    

---

For every hour, the dataset reports the following variables:

1. Index of record: `instant`
2. Day of month: `day`
3. Season: `season (1: spring, 2: summer, 3: fall, 4: winter)`
4. Year: `yr (0: 2011, 1: 2012)`
5. Month: `mnth (1 to 12)`
6. Hour: `hr (0 to 23)`
7. Holiday status: `holiday`
8. Day of the week: `weekday`
9. Working day status: `workingday`
10. Weather situation: `weathersit (1: clear, 2:mist, 3: light rain/snow, 4: heavy rain/snow)`
11. Temperature in °C: `temp`
12. Perceived temperature in °C: `atemp`
13. Humidity: `hum`
14. Wind speed: `windspeed`
15. Number of casual users: `casual`
16. Number of registered users: `registered`
17. Count of rental bikes: `cnt`


In a time series dataset such as this one, rows represent successive time-points: there is a dimension along which they are ordered. Sure, we could treat each row as independent and try to predict the number of circulating bikes based on, say, a particular time of day regardless of what happened earlier. However, the existence of an ordering gives us the opportunity to exploit causal relationships across time. For instance, it allows us to predict bike rides at one time based on the fact that it was raining at an earlier time. For the time being, we’re going to focus on learning how to turn our bike-sharing dataset into something that our neural network will be able to ingest in fixed-size chunks.

This neural network model will need to see a number of sequences of values for
each different quantity, such as ride count, time of day, temperature, and weather conditions: N parallel sequences of size C. C stands for channel, in neural network parlance, and is the same as column for 1D data like we have here. The N dimension represents the time axis, here one entry per hour.

---

<h2> <center> <b> <u> Shaping the Data by Time Period </u> </b> </center> </h2>

We might want to break up the two-year dataset into wider observation periods, like days. This way we’ll have N (for number of samples) collections of C sequences of length L. In other words, our time series dataset would be a tensor of dimension 3 and shape N × C × L. The C would remain our 17 channels, while L would be 24: 1 per hour of the day. There’s no particular reason why we must use chunks of 24 hours, though the general daily rhythm is likely to give us patterns we can exploit for predictions. We could also use 7 × 24 = 168 hour blocks to chunk by week instead, if we desired. All of this depends, naturally, on our dataset having the right size—the number of rows must be a multiple of 24 or 168. Also, for this to make sense, we cannot have gaps in the time series.


Let’s go back to our bike-sharing dataset. The first column is the index (the global ordering of the data), the second is the date, and the sixth is the time of day. We have everything we need to create a dataset of daily sequences of ride counts and other exogenous variables. Our dataset is already sorted, but if it were not, we could use `torch.sort` on it to order it appropriately.

  **[IMPORTANT]** NOTE The version of the file we’re using, hour-fixed.csv, has had some processing done to include rows missing from the original dataset. We presume that the missing hours had zero bike active (they were typically in the early morning hours).

All we have to do to obtain our daily hours dataset is view the same tensor in batches of 24 hours. Let’s take a look at the shape and strides of our `bikes` tensor:

In [2]:
print('Bike Shape: {} | Bike Stride: {}'.format(bikes.shape, bikes.stride()))

Bike Shape: torch.Size([17520, 17]) | Bike Stride: (17, 1)


---

That’s 17,520 hours, 17 columns. Now let’s reshape the data to have 3 axes—day, hour, and then our 17 columns:

---

In [3]:
daily_bikes = bikes.view(-1, 24, bikes.shape[1])
print('Bike Shape: {} | Bike Stride: {}'.format(daily_bikes.shape, daily_bikes.stride()))

Bike Shape: torch.Size([730, 24, 17]) | Bike Stride: (408, 17, 1)


---
Some points about our above written code lines:

- `bikes.shape[1]` is `17`, the number of columns in the bikes tensor.
- `view` changes the way the tensor looks at the same data as contained in storage.

---

Remember that storage is a contiguous, linear container for numbers. Our bikes tensor will have each row stored one after the other in its corresponding storage. This is confirmed by the output from the call to `bikes.stride()` earlier.

For `daily_bikes`, the stride is telling us that advancing by 1 along the hour dimension (the second dimension) requires us to advance by 17 places in the storage (or one set of columns); whereas advancing along the day dimension (the first dimension) requires us to advance by a number of elements equal to the length of a row in the storage times 24 (here, 408, which is 17 × 24).

We see that the rightmost dimension is the number of columns in the original
dataset. Then, in the middle dimension, we have time, split into chunks of 24 sequential hours. In other words, we now have N sequences of L hours in a day, for C channels. To get to our desired N × C × L ordering, we need to transpose the tensor:



In [4]:
daily_bikes = daily_bikes.transpose(1, 2)
print('Bike Shape: {} | Bike Stride: {}'.format(daily_bikes.shape, daily_bikes.stride()))

Bike Shape: torch.Size([730, 17, 24]) | Bike Stride: (408, 1, 17)


---

<h2> <center> <b> <u> Ready for Training </u> </b> </center> </h2>

The “weather situation” variable is ordinal. It has four levels: 1 for good weather, and 4 for, really bad. We could treat this variable as categorical, with levels interpreted as labels, or as a continuous variable. If we decided to go with categorical, we would turn the variable into a one-hot-encoded vector and concatenate the columns with the dataset.

In order to make it easier to render our data, we’re going to limit ourselves to the first day for a moment. We initialize a zero-filled matrix with a number of rows equal to the number of hours in the day and number of columns equal to the number of weather levels:


In [5]:
first_day = bikes[:24].long()
weather_onehot = torch.zeros(first_day.shape[0], 4)
first_day[:,9]

tensor([1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 2, 2, 2, 2])

---

Then we scatter ones into our matrix according to the corresponding level at each row. Remember the use of `unsqueeze` to add a singleton dimension as we did in the previous sections:

In [6]:
weather_onehot.scatter_(dim = 1, index = first_day[:,9].unsqueeze(1).long() - 1,
                        value = 1.0)

tensor([[1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.]])

We have used `-1` in `index = first_day[:,9].unsqueeze(1).long() - 1` because weather situation range from 1 to 4, while indices are 0-based.

Last, we concatenate our matrix to our original dataset using the `cat` function. Let’s look at the first of our results:

In [7]:
torch.cat((bikes[:24], weather_onehot), 1)[:1]

tensor([[ 1.0000,  1.0000,  1.0000,  0.0000,  1.0000,  0.0000,  0.0000,  6.0000,
          0.0000,  1.0000,  0.2400,  0.2879,  0.8100,  0.0000,  3.0000, 13.0000,
         16.0000,  1.0000,  0.0000,  0.0000,  0.0000]])

---

Here we prescribed our original `bikes` dataset and our one-hot-encoded “weather situation” matrix to be concatenated along the `column` dimension (that is, 1). In other words, the columns of the two datasets are stacked together; or, equivalently, the new one-hot-encoded columns are appended to the original dataset. For `cat` to succeed, it is required that the tensors have the same size along the other dimensions — the row dimension, in this case. Note that our new last four columns are 1, 0, 0, 0, exactly as we would expect with a weather value of 1.

We could have done the same with the reshaped `daily_bikes` tensor. Remember
that it is shaped $(B, C, L)$, where $L = 24$. We first create the zero tensor, with the same $B$ and $L$, but with the number of additional columns as $C$:

In [8]:
daily_weather_onehot = torch.zeros(daily_bikes.shape[0], 4, 
                                   daily_bikes.shape[2])

print('Daily Weather Onehot Shape: {}'.format(
    daily_weather_onehot.shape
))

Daily Weather Onehot Shape: torch.Size([730, 4, 24])


---

Then we scatter the one-hot encoding into the tensor in the C dimension. Since this operation is performed in place, only the content of the tensor will change:

In [9]:
daily_weather_onehot.scatter_(1, daily_bikes[:,9,:].long().unsqueeze(1)-1, 1.0)
print('Daily Weather Onehot Shape: {}'.format(daily_weather_onehot.shape))

Daily Weather Onehot Shape: torch.Size([730, 4, 24])


---

We concatenate along the $C$ dimension:

----

In [10]:
daily_bikes = torch.cat((daily_bikes, daily_weather_onehot), dim = 1)

---

We mentioned earlier that this is not the only way to treat our ``weather situation'' variable. Indeed, its labels have an ordinal relationship, so we could pretend they are special values of a continuous variable. We could just transform the variable so that it runs from 0.0 to 1.0:

---

In [11]:
daily_bikes[:, 9, :] = (daily_bikes[:, 9,:] - 1.0)/3.0

---

As we mentioned in the previous section, rescaling variables to the [0.0, 1.0] interval or the [-1.0, 1.0] interval is something we’ll want to do for all quantitative variables, like temperature (column 10 in our dataset). We’ll see why later; for now, let’s just say that this is beneficial to the training process.

There are multiple possibilities for rescaling variables. We can either map their range to [0.0, 1.0]

---

In [12]:
temp = daily_bikes[:, 10, :]
temp_min = torch.min(temp)
temp_max = torch.max(temp)
daily_bikes[:, 10, :] = ((daily_bikes[:, 10, :] - temp_min)/
                         (temp_max - temp_min))

---

or subtract the mean and divide by the standard deviation:

----

In [13]:
temp = daily_bikes[:, 10, :]
daily_bikes[:, 10, :] = ((daily_bikes[:, 10, :] - torch.mean(temp))/
                         torch.std(temp))

---

In the latter case, our variable will have 0 mean and unitary standard deviation. If our variable were drawn from a Gaussian distribution, 68% of the samples would sit in the [-1.0, 1.0] interval.

**Great:** we’ve built another nice dataset, and we’ve seen how to deal with time series data. For this tour d’horizon, it’s important only that we got an idea of how a time series is laid out and how we can wrangle the data in a form that a network will digest.

Other kinds of data look like a time series, in that there is a strict ordering. Top two on the list? Text and audio. We’ll take a look at text in the next module.

----